In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy
import requests
from WalkScore import *
import copy
from create_date_features import create_date_features



Unnamed: 0                                500324
dropoff_latitude                          40.774
dropoff_longitude                       -73.8709
extra                                          0
fare_amount                                   31
improvement_surcharge                        0.3
mta_tax                                      0.5
passenger_count                                2
payment_type                                   1
pickup_latitude                          40.7569
pickup_longitude                        -73.9731
ratecodeid                                     1
store_and_fwd_flag                             N
tip_amount                                     5
tolls_amount                                5.54
total_amount                               42.34
tpep_dropoff_datetime    2016-04-12T10:28:02.000
tpep_pickup_datetime     2016-04-12T10:05:28.000
trip_distance                               10.7
vendorid                                       1
Name: 0, dtype: obje

In [2]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
from create_weather_features import create_weather_features

2016 1 1
2016 1 2
2016 1 3
2016 1 4
2016 1 5
2016 1 6
2016 1 7
2016 1 8
2016 1 9
2016 1 10
2016 1 11
2016 1 12
2016 1 13
2016 1 14
2016 1 15
2016 1 16
2016 1 17
2016 1 18
2016 1 19
2016 1 20
2016 1 21
2016 1 22
2016 1 23
2016 1 24
2016 1 25
2016 1 26
2016 1 27
2016 1 28
2016 1 29
2016 1 30
2016 1 31
2016 2 1
2016 2 2
2016 2 3
2016 2 4
2016 2 5
2016 2 6
2016 2 7
2016 2 8
2016 2 9
2016 2 10
2016 2 11
2016 2 12
2016 2 13
2016 2 14
2016 2 15
2016 2 16
2016 2 17
2016 2 18
2016 2 19
2016 2 20
2016 2 21
2016 2 22
2016 2 23
2016 2 24
2016 2 25
2016 2 26
2016 2 27
2016 2 28
2016 2 29
2016 3 1
2016 3 2
2016 3 3
2016 3 4
2016 3 5
2016 3 6
2016 3 7
2016 3 8
2016 3 9
2016 3 10
2016 3 11
2016 3 12
2016 3 13
2016 3 14
2016 3 15
2016 3 16
2016 3 17
2016 3 18
2016 3 19
2016 3 20
2016 3 21
2016 3 22
2016 3 23
2016 3 24
2016 3 25
2016 3 26
2016 3 27
2016 3 28
2016 3 29
2016 3 30
2016 3 31
2016 4 1
2016 4 2
2016 4 3
2016 4 4
2016 4 5
2016 4 6
2016 4 7
2016 4 8
2016 4 9
2016 4 10
2016 4 11
2016 4 12
2016 4

In [3]:
cab_data_p1 = pd.read_csv("cab_data_p1.csv")
cab_data_p2 = pd.read_csv("cab_data_p2.csv")
cab_data = pd.read_csv("cab_data.csv")

In [4]:
#Removes columns we dont need, feel free to change this based on what features we want/need
def scrub_df(df):
    del df['extra']
    del df['fare_amount']
    del df['improvement_surcharge']
    del df['mta_tax']
    del df['passenger_count']
    del df['payment_type']
    del df['ratecodeid']
    del df['store_and_fwd_flag']
    del df['tip_amount']
    del df['tolls_amount']
    del df['trip_distance']
    del df['vendorid']
    
    return df

In [5]:
#Clean the input dataset of columns we don't care about
basic_df = scrub_df(cab_data)
basic_df.to_csv('clean_cab_data.csv')
basic_df_p1 = scrub_df(cab_data_p1)
basic_df_p2 = scrub_df(cab_data_p2)

In [6]:
#Pairs the lat/lon columns into one column (for walkscore/geolocator)
def get_latlng(df):
    def f(x,y):
        coord = "%f, %f" % (x, y)
        return coord
    
    df['start_latlng'] = df[['dropoff_latitude', 'dropoff_longitude']].apply(lambda x: f(*x), axis=1)
    df['end_latlng'] = df[['pickup_latitude', 'pickup_longitude']].apply(lambda x: f(*x), axis=1)
    
    return df

df_with_latlng = get_latlng(basic_df)

In [7]:
#REMOVE WHEN UPLOADING
#apiKey = 
#walkscore = WalkScore(apiKey)
geolocator = Nominatim()

In [8]:
#Given input dataframe, computes distance of trip using geolocator and start/end lat/lon positions
def get_dist(df):

    def get_d(x,y):
        return geodesic(x, y).miles

    df['distance'] = df[['start_latlng', 'end_latlng']].apply(lambda x: get_d(*x), axis=1)
    
    return df

df_with_dist = get_dist(df_with_latlng)

In [9]:
df_with_date = create_date_features(df_with_dist)
#print(df_with_date)
df_with_date.to_csv("date_data.csv")

In [10]:
open_date_df = pd.read_csv("date_data.csv")
df_with_weather = create_weather_features(open_date_df)
#print(df_with_weather)

In [16]:
#For now, write to CSV file, once we figure out how to make enough geo queries, add walkscore feature
#print(df_with_dist)
df_with_weather.to_csv('data_with_features.csv')
df_small = df_with_dist[:100]
print(df_small)
import random

    Unnamed: 0  dropoff_latitude  dropoff_longitude  pickup_latitude  \
0            0         40.777241         -73.951744        40.776424   
1            1         40.777611         -73.951080        40.789371   
2            2         40.770386         -73.966476        40.761303   
3            3         40.767288         -73.966675        40.764381   
4            4         40.744068         -73.988991        40.754742   
5            5         40.787369         -73.954048        40.771824   
6            6         40.777973         -73.955894        40.759167   
7            7         40.737041         -73.988632        40.746235   
8            8         40.756405         -73.968155        40.761028   
9            9         40.805321         -73.961723        40.819996   
10          10         40.714500         -73.992882        40.725052   
11          11         40.755898         -73.975319        40.748158   
12          12         40.752850         -73.996689        40.76

In [19]:
#SLOW
import time
global count

def get_address(df):
    count = 0
    def reverse_geocode(latlng):
        global count
        
        geo = random.choice([geolocator, geolocator])
        res = geo.reverse(latlng)
        #print(count)
        #print(res)
        print("done!")
        #count += 1
        time.sleep(.1)
        return res.address
    
    #VERY SLOW
    df['pickup_address'] = df['start_latlng'].apply(reverse_geocode)
    df['dropoff_address'] = df['end_latlng'].apply(reverse_geocode)    
    
    return df

df_with_address = get_address(df_small)

done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!


GeocoderTimedOut: Service timed out

In [ ]:
#Given part 1 and part 2 of data, returns a DF of walkability score for 
#Start and end coord
#SLOW
def get_walkability_score(df):
    def get_scores(base, addrs):
        scores_dict = dict()
        for addr in addrs:
            a = requests.get(base + addr)
            root = BeautifulSoup(a.text, "html5lib")
            scores = root.find_all("div", class_="block-header-badge score-info-link")
            scores_dict[addr] = dict()
            for i in range(len(scores)):
                img_tag = scores[i].find("img")['src']
                split_tags = img_tag.split('/')
                c_score = split_tags[-1]
                c_score = int(c_score[:len(c_score)-4])

                if ("walk/score" in img_tag):
                    scores_dict[addr]['walk'] = c_score
                elif ("bike/score" in img_tag):
                    scores_dict[addr]['bike'] = c_score
                elif ("transit/score" in img_tag):
                    scores_dict[addr]['transit'] = c_score
                else:
                    print("Invalid score")
        return scores_dict
    
    def ws_score(lat,lon, address):
        #score = walkscore.makeRequest(address, lat, lon)
        base = "https://www.walkscore.com/score/"
        scores_dict = get_scores(base, [address])
        
        print(scores_dict)
        return scores_dict['walk']
    
    df['pickup_addr'] = df[['pickup_latitude', 'pickup_longitude', 'pickup_address']].apply(lambda x: ws_score(*x), axis=1)
    df['dropoff_addr'] = df[['dropoff_latitude', 'dropoff_longitude', 'dropoff_address']].apply(lambda x: ws_score(*x), axis=1)
    
    return df

In [35]:
print("Getting addresses")
df_with_walkscore = get_walkability_score(df_with_address)

Getting addresses


KeyError: "['pickup_address'] not in index"